In [ ]:
!pip install fastapi uvicorn transformers torch nest_asyncio pyngrok

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import nest_asyncio
from pyngrok import ngrok

# Initialize the FastAPI app
app = FastAPI()

# Load your fine-tuned model from Colab
model_path = "./fine_tuned_darijaBERT"  # Path where your fine-tuned model is stored
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Define sentiment analysis pipeline
sentiment_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Define request/response models
class SentimentRequest(BaseModel):
    text: str

class SentimentResponse(BaseModel):
    label: str
    score: float

# Endpoint for sentiment analysis
@app.post("/analyze", response_model=SentimentResponse)
async def analyze_sentiment(request: SentimentRequest):
    try:
        result = sentiment_pipeline(request.text)[0]
        return SentimentResponse(label=result["label"], score=result["score"])
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Start the FastAPI server using ngrok
def start_ngrok():
    url = ngrok.connect(8000).public_url
    print(f"Public URL: {url}")

# Start the FastAPI server
def start_server():
    import uvicorn
    nest_asyncio.apply()
    start_ngrok()
    uvicorn.run(app, host="0.0.0.0", port=8000)

start_server()

In [ ]:
async function analyzeSentiment() {
  const text = document.getElementById('input-text').value;
  const resultDiv = document.getElementById('result');
  resultDiv.innerHTML = "Analyzing...";

  try {
    const response = await fetch('https://xyz.ngrok.io/analyze', {  // Replace with your ngrok URL
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ text })
    });
    const data = await response.json();
    resultDiv.innerHTML = `<p>Label: <b>${data.label}</b></p><p>Score: <b>${data.score.toFixed(2)}</b></p>`;
  } catch (error) {
    resultDiv.innerHTML = "<p style='color: red;'>Error analyzing sentiment</p>";
  }
}

### **Test the Interface**

1. Open the `index.html` file in your browser.
2. Enter the text and click **Analyze**. It will send a request to the FastAPI backend running in Colab.
3. The response (sentiment label and score) will be displayed.

### **Persist the Backend**

If you need to keep the backend running for an extended period:

- Keep Colab running in the background.
- Alternatively, deploy your model and backend to platforms like **AWS**, **Heroku**, or **Render**.